In [ ]:
        try: 
            from .exporters import CodeExporter
            from .markdown import IndentInline
        except:
            from exporters import CodeExporter
            from markdown import IndentInline
        from IPython.display import display, Markdown

        from nbformat import NotebookNode
        from nbformat.v4 import new_code_cell, new_notebook
        from nbconvert import export
        from types import MethodType

        from IPython.core.interactiveshell import InteractiveShell, ExecutionResult
        __all__ = []
        exporter = CodeExporter()

In [ ]:
        class Repl:
            def __init__(self, kernel):
                self.kernel = kernel
                
            def read(self, text):
                return text
            
            def eval(self, text):
                return new_notebook(cells=[new_code_cell(source=text)])            

            def print(*args, **kwargs): ...

            def run_cell(self, content, *args, **kwargs):
                if not isinstance(content, NotebookNode):
                    content = self.read(content)
                node = self.eval(content)
                result = self.loop(node, *args, **kwargs)
                return self.print(node) or result

In [ ]:
        def run_node(kernel, node, store_history, silent, shell_futures):
            code = export(exporter, node)[0][1:]
            return InteractiveShell.run_cell(kernel, code, store_history=store_history, silent=silent, shell_futures=shell_futures)

In [ ]:
        class RunCell(Repl):                
            def loop(self, node, store_history=False, silent=False, shell_futures=True):
                return run_node(self.kernel, node, store_history, silent, shell_futures)

In [ ]:
        class Literate(RunCell):
            def read(self, text):
                display(Markdown(text))
                return text

In [ ]:
        def repl(cls):
            def _load_ipython_extension(ip=__import__("IPython").get_ipython()):
                ip.run_cell = cls(ip).run_cell
            return _load_ipython_extension

In [ ]:
        load_ipython_extension = repl(Literate)

        def unload_ipython_extension(ip=__import__('IPython').get_ipython()):
            ip.run_cell = MethodType(InteractiveShell.run_cell, ip)